This notebook calculates the change in direct damage to specified assets

In [11]:
# Imports
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
# import datetime
from ci_adapt_utilities import *
import pickle

In [12]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [ ]:
# Function Definitions


In [13]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets = gpd.read_file(assets_path)
assets = gpd.GeoDataFrame(assets).set_crs(4326).to_crs(3857)
assets = assets.loc[assets.geometry.geom_type == 'LineString']
assets = assets.rename(columns={'railway' : 'asset'})

# # Drop bridges and tunnels
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]
# assets = assets.loc[~(assets['tunnel'].isin(['yes']))]

assets = assets[assets['railway:traffic_mode']!='"passenger"']
assets = assets.reset_index(drop=True)

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

# Read vulnerability and maximum damage data from Nirandjan, S., et al. (2024)
curve_types = {'rail': ['F8.1']}
infra_curves, maxdams = ds.read_vul_maxdam(data_path, hazard_type, infra_type)
max_damage_tables = pd.read_excel(data_path / vulnerability_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
print(f'Found matching infrastructure curves for: {infra_type}')

3187 assets loaded.
Found matching infrastructure curves for: rail


In [28]:
# create a variable changed_assets
changed_asset_list = [37,314,3004] # testing with a few assets

# open pickled hazard-asset overlay and hazard intensity data
with open(interim_data_path / 'overlay_assets_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    overlay_assets = pickle.load(f)
with open(interim_data_path / 'hazard_numpified_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    hazard_numpified_list = pickle.load(f)


In [29]:

# Set up the path
adapt_path = Path(r'C:\Data\interim\adaptations\test_haz_level_adapt.geojson')

# Load the polygon of the protected area
protected_area = gpd.read_file(adapt_path).to_crs(3857)

# Filter assets that are within the protected area
filtered_assets = gpd.overlay(assets, protected_area, how='intersection')
f_assets = assets.loc[(assets['osm_id'].isin(filtered_assets['osm_id']))]
f_assets.head(3)
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]


,osm_id,asset,name,gauge,electrified,voltage,bridge,maxspeed,service,tunnel,other_tags,railway:traffic_mode,usage,geometry,buffered
37,7990468,rail,None,1435,contact_line,15000,None,120,None,None,"""frequency""=>""16.7"",""operator""=>""DB Netz AG"",""...","""mixed""","""main""","LINESTRING (810131.824 6545141.628, 810131.034...","POLYGON ((810131.0332578894 6545143.573510949,..."
313,33506714,rail,Rechte Rheinstrecke,1435,contact_line,15000,yes,100,None,None,"""frequency""=>""16.7"",""layer""=>""1"",""operator""=>""...","""mixed""","""main""","LINESTRING (806342.732 6547553.449, 806364.083...","POLYGON ((806364.0829647976 6547544.350868201,..."
314,33506715,rail,Rechte Rheinstrecke,1435,contact_line,15000,None,100,None,None,"""embankment""=>""yes"",""frequency""=>""16.7"",""opera...","""mixed""","""main""","LINESTRING (806681.210 6547474.401, 806676.935...","POLYGON ((806676.934860112 6547474.821351372, ..."


In [30]:

# optionally to calculate the maximum intensity for each hazard point, this can be used, else a float can be used
max_intensity = []
for asset_id in changed_asset_list:
    max_intensity.append(retrieve_max_intensity_by_asset(asset_id, overlay_assets, hazard_numpified_list))


In [33]:

# extract the assets that will be adapted
changed_assets = assets.loc[assets.index.isin(changed_asset_list)].copy() # testing with a few assets
# add new columns fragility_mod and haz_mod
changed_assets['fragility_mod'] = 1 #[0.3, 0.5, 0.8] #fraction [example considering no reduction] (1 = no reduction, 0 = invulnerable asset) DUMMY DATA FOR TESTING
changed_assets['haz_mod'] = [np.max(x) for x in max_intensity] #meters [example adding wall of maximum flooding depth + 1 meter] (0 = no reduction in hazard intensity, 0.5 = 0.5 meter reduction in hazard intensity) DUMMY DATA FOR TESTING consider raising railway 0.5 meters

# TODO: automate infrastructure curve deduction from dictionary keys, now running with curve F8.1
hazard_intensity = infra_curves['F8.1'].index.values
fragility_values = (np.nan_to_num(infra_curves['F8.1'].values,nan=(np.nanmax(infra_curves['F8.1'].values)))).flatten()
maxdams_filt=max_damage_tables[max_damage_tables['ID number']=='F8.1']['Amount']

adaptation_run = run_damage_reduction_by_asset(geom_dict, overlay_assets, hazard_numpified_list, changed_assets, hazard_intensity, fragility_values, maxdams_filt)

#reporting
for asset_id, baseline_damages in adaptation_run[0].items():
    print(f'\nADAPTATION results for asset {asset_id}:')
    print(f'Baseline damages for asset {asset_id}: {baseline_damages[0]:.2f} to {baseline_damages[1]:.2f} EUR')
    print(f'Adapted damages for asset {asset_id}: {adaptation_run[1][asset_id][0]:.2f} to {adaptation_run[1][asset_id][1]:.2f} EUR')
    delta = tuple(adaptation_run[1][asset_id][i] - baseline_damages[i] for i in range(len(baseline_damages)))
    # percent_change = tuple((100 * (delta[i] / baseline_damages[i])) for i in range(len(baseline_damages)))
    percent_change = tuple((100 * (delta[i] / baseline_damages[i])) if baseline_damages[i] != 0 else 0 for i in range(len(baseline_damages)))
    print(f'Change (Adapted-Baseline): {delta[0]:.2f} to {delta[1]:.2f} EUR, {percent_change}% change, at a cost of {adaptation_run[2][asset_id]:.2f} EUR')

#TODO Check with economist: ammortization of adaptation cost over years of adaptation scenario
    #NEXT: adaptation_run returns (collect_inb_bl, collect_inb_adapt, adaptation_cost). These can be used to calculate the EAD for the adapted scenario (and damage reduction), and compare with the adaptation cost, which must be ammortized over the years of the adaptation scenario.




2024-06-17 10:47:55 - Calculating adapted damages for assets...


100%|██████████| 98/98 [00:02<00:00, 45.74it/s]

95 assets with no change.

ADAPTATION results for asset 37:
Baseline damages for asset 37: 543666.18 to 2458653.92 EUR
Adapted damages for asset 37: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -543666.18 to -2458653.92 EUR, (-100.0, -100.0)% change, at a cost of 22101136.68 EUR

ADAPTATION results for asset 314:
Baseline damages for asset 314: 0.00 to 112136.25 EUR
Adapted damages for asset 314: 0.00 to 0.00 EUR
Change (Adapted-Baseline): 0.00 to -112136.25 EUR, (0, -100.0)% change, at a cost of 652732.03 EUR

ADAPTATION results for asset 3004:
Baseline damages for asset 3004: 355376.38 to 355376.38 EUR
Adapted damages for asset 3004: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -355376.38 to -355376.38 EUR, (-100.0, -100.0)% change, at a cost of 3194521.13 EUR
